<a href="https://colab.research.google.com/github/Jennamui/AutoML-Examples/blob/main/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages


In [1]:
!pip install tpot mljar-supervised

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.4 MB/s 
     |████████████████████████████████| 112 kB 41.2 MB/s 
     |████████████████████████████████| 139 kB 49.7 MB/s 
     |████████████████████████████████| 255.9 MB 38 kB/s 
     |████████████████████████████████| 2.0 MB 36.7 MB/s 
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 72 kB 929 kB/s 
     |████████████████████████████████| 569 kB 51.2 MB/s 
     |████████████████████████████████| 72 kB 909 kB/s 
     |████████████████████████████████| 348 kB 47.9 MB/s 
     |████████████████████████████████| 81 kB 7.5 MB/s 
     |████████████████████████████████| 209 kB 47.1 MB/s 
     |████████████████████████████████| 78 kB 4.5 MB/s 
     |████████████████████████████████| 147 kB 57.0 MB/s 
     |████████████████████████████████| 112 kB 48.1 MB/s 
     |████████████████████████████████| 5

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML

import numpy as np


In [ ]:
#automl = AutoML(
    # mode="Explain"
    # algorithms=""
    # results_path="AutoML_22",
    # total_time_limit=30 * 60,
    # train_ensemble=True,
    # stack_models="",
    # eval_metric=""
#)

# Healthcare Dataset - SPARCS

## Load in dataset

In [2]:
import pandas as pd
sparcs = pd.read_csv('https://raw.githubusercontent.com/hantswilliams/HHA-507-2022/main/autoML/datasets/data_sparcs.csv')
sparcs

,Health Service Area,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
0,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,30 to 49,147,M,White,Not Span/Hispanic,...,Minor,Medical,Private Health Insurance,NaN,NaN,0,N,Y,4757.01,4747.83
1,Western NY,Allegany,228000.0,39.0,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,18 to 29,148,F,White,Not Span/Hispanic,...,Minor,Medical,Blue Cross/Blue Shield,Self-Pay,Self-Pay,0,N,N,5090.25,2985.64
2,Western NY,Allegany,228000.0,39.0,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,0 to 17,147,M,White,Not Span/Hispanic,...,Minor,Medical,Self-Pay,Self-Pay,Self-Pay,2900,N,N,4948.50,2129.67
3,Western NY,Allegany,228000.0,39.0,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,70 or Older,148,F,White,Not Span/Hispanic,...,Moderate,Medical,Medicare,Medicare,Self-Pay,0,N,Y,4719.75,8454.41
4,Western NY,Allegany,228000.0,39.0,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,50 to 69,148,M,White,Not Span/Hispanic,...,Major,Medical,Blue Cross/Blue Shield,Medicare,Self-Pay,0,N,Y,50384.75,34565.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23578,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,70 or Older,117,F,White,Not Span/Hispanic,...,Moderate,Medical,Medicare,Private Health Insurance,NaN,0,N,Y,50833.00,8961.40
23579,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,0 to 17,117,F,Other Race,Spanish/Hispanic,...,Minor,Medical,Private Health Insurance,NaN,NaN,3200,N,N,10948.00,2214.06
23580,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,30 to 49,117,M,White,Not Span/Hispanic,...,Minor,Medical,Medicaid,NaN,NaN,0,N,N,46421.00,11083.24
23581,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,70 or Older,117,M,White,Not Span/Hispanic,...,Major,Medical,Medicare,Medicare,NaN,0,N,Y,46122.00,7951.26


In [3]:
sparcs.columns

Index(['Health Service Area', 'Hospital County',
       'Operating Certificate Number', 'Facility Id', 'Facility Name',
       'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race', 'Ethnicity',
       'Length of Stay', 'Type of Admission', 'Patient Disposition',
       'Discharge Year', 'CCS Diagnosis Code', 'CCS Diagnosis Description',
       'CCS Procedure Code', 'CCS Procedure Description', 'APR DRG Code',
       'APR DRG Description', 'APR MDC Code', 'APR MDC Description',
       'APR Severity of Illness Code', 'APR Severity of Illness Description',
       'APR Risk of Mortality', 'APR Medical Surgical Description',
       'Payment Typology 1', 'Payment Typology 2', 'Payment Typology 3',
       'Birth Weight', 'Abortion Edit Indicator',
       'Emergency Department Indicator', 'Total Charges', 'Total Costs'],
      dtype='object')

In [4]:
x_cols = [c for c in sparcs.columns if c != "Total Charges"]
X = sparcs[x_cols]
y = sparcs["Total Charges"]

In [6]:
x_cols

['Health Service Area',
 'Hospital County',
 'Operating Certificate Number',
 'Facility Id',
 'Facility Name',
 'Age Group',
 'Zip Code - 3 digits',
 'Gender',
 'Race',
 'Ethnicity',
 'Length of Stay',
 'Type of Admission',
 'Patient Disposition',
 'Discharge Year',
 'CCS Diagnosis Code',
 'CCS Diagnosis Description',
 'CCS Procedure Code',
 'CCS Procedure Description',
 'APR DRG Code',
 'APR DRG Description',
 'APR MDC Code',
 'APR MDC Description',
 'APR Severity of Illness Code',
 'APR Severity of Illness Description',
 'APR Risk of Mortality',
 'APR Medical Surgical Description',
 'Payment Typology 1',
 'Payment Typology 2',
 'Payment Typology 3',
 'Birth Weight',
 'Abortion Edit Indicator',
 'Emergency Department Indicator',
 'Total Costs']

In [7]:
X

,Health Service Area,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Costs
0,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,30 to 49,147,M,White,Not Span/Hispanic,...,Minor,Minor,Medical,Private Health Insurance,NaN,NaN,0,N,Y,4747.83
1,Western NY,Allegany,228000.0,39.0,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,18 to 29,148,F,White,Not Span/Hispanic,...,Minor,Minor,Medical,Blue Cross/Blue Shield,Self-Pay,Self-Pay,0,N,N,2985.64
2,Western NY,Allegany,228000.0,39.0,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,0 to 17,147,M,White,Not Span/Hispanic,...,Minor,Minor,Medical,Self-Pay,Self-Pay,Self-Pay,2900,N,N,2129.67
3,Western NY,Allegany,228000.0,39.0,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,70 or Older,148,F,White,Not Span/Hispanic,...,Moderate,Moderate,Medical,Medicare,Medicare,Self-Pay,0,N,Y,8454.41
4,Western NY,Allegany,228000.0,39.0,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,50 to 69,148,M,White,Not Span/Hispanic,...,Extreme,Major,Medical,Blue Cross/Blue Shield,Medicare,Self-Pay,0,N,Y,34565.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23578,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,70 or Older,117,F,White,Not Span/Hispanic,...,Moderate,Moderate,Medical,Medicare,Private Health Insurance,NaN,0,N,Y,8961.40
23579,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,0 to 17,117,F,Other Race,Spanish/Hispanic,...,Minor,Minor,Medical,Private Health Insurance,NaN,NaN,3200,N,N,2214.06
23580,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,30 to 49,117,M,White,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,0,N,N,11083.24
23581,Long Island,Suffolk,5157003.0,943.0,St Catherine of Siena Hospital,70 or Older,117,M,White,Not Span/Hispanic,...,Major,Major,Medical,Medicare,Medicare,NaN,0,N,Y,7951.26


In [8]:
y

0         4757.01
1         5090.25
2         4948.50
3         4719.75
4        50384.75
           ...   
23578    50833.00
23579    10948.00
23580    46421.00
23581    46122.00
23582    32225.00
Name: Total Charges, Length: 23583, dtype: float64

In [11]:
automl = AutoML()
automl.fit(X, y)

Linear algorithm was disabled.
AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline rmse 72381.653229 trained in 0.49 seconds


2_DecisionTree rmse 33806.950052 trained in 12.74 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost rmse 19944.100875 trained in 153.22 seconds
4_Default_NeuralNetwork rmse 36631.607976 trained in 7.61 seconds
5_Default_RandomForest rmse 36887.793213 trained in 18.17 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 18837.918818 trained in 0.3 seconds
AutoML fit time: 201.44 seconds
AutoML best model: Ensemble


AutoML()

In [12]:
sparcs["predictions"] = automl.predict(X)


In [13]:
print("Predictions")
print(sparcs[["Total Charges", "predictions"]].head())

Predictions
   Total Charges   predictions
0        4757.01   7439.961726
1        5090.25   7661.853523
2        4948.50   7600.419929
3        4719.75   7525.814851
4       50384.75  58561.460695


# Download outputs

In [18]:
# get current working directory
import os
os.getcwd()

'/content'

In [23]:
!zip -r /content/AutoML_1.zip /content/AutoML_1

  adding: content/AutoML_1/ (stored 0%)
  adding: content/AutoML_1/data_info.json (deflated 84%)
  adding: content/AutoML_1/ldb_performance_boxplot.png (deflated 22%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/ (stored 0%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/status.txt (stored 0%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/permutation_importance.png (deflated 15%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/learner_fold_0_importance.csv (deflated 46%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/true_vs_predicted.png (deflated 8%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/README.md (deflated 51%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/predicted_vs_residuals.png (deflated 8%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/learner_fold_0.neural_network (deflated 0%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/learner_fold_0_training.log (deflated 53%)
  adding: content/AutoML_1/4_Default_NeuralNetwork/framework.j